<center> <b>Geospatial Fellows Program</b> </center>


# Identifying High Accuracy Regions in Traffic Camera Images to Enhance the Estimation of Road Traffic Metrics: A Quadtree-based Method and Applications 
<i>Ningchuan Xiao, Yue Lin</i>

This Jupyter notebook illustrates a quadtree-based algorithm to establish the image regions with high vehicle detection accuracy. A case study is presented to demonstrate how the use of these high accuracy regions can produce reliable road traffic metrics from traffic camera images.

## Notebook Outline
- [Introduction](#intro)
- [Setup](#setup)
- [Vehicle detector](#det)    
- [References](#ref)

<a id='intro'></a>
## Introduction
Traffic cameras have become an important data source for transportation management and control. However, deriving reliable traffic metrics from the camera feeds is challenging due to the limitations of current vehicle detection techniques, as well as the various camera conditions such as height and resolution. The figure below shows two examples of using a state-of-the-art deep learning detector to identify the vehicles in traffic images. It is clear that vehicles distant from the camera are often too small to be detected. Missed small vehicles tend to result in significant errors in the estimation of road traffic metrics. 

<img src='./figs/missed.png' width='800'/>

In this jupyter notebook, we illustrate a quadtree-based algorithm that continuously partitions the image extent until only regions with high detection accuracy are remained. These regions are referred to as the high-accuracy identification regions (HAIR), which will be used to derive reliable road traffic metrics such as traffic density, flow, and speed. A case study in Central Ohio is presented to demonstrate how the use of HAIR can help to improve the accuracy of traffic density estimates using images from a camera. 

<a id='setup'></a>
## Setup

We load the necessary libraries in the first cell.

<a id='det'></a>
## Vehicle detector

The use of HAIR assumes some object detector is available to identify the vehicles present in the images. In this project, a deep learning model called EfficientDet[<sup>1</sup>](#fn1) is used for vehicle identification. Two data sets are prepared using the camera feeds in Central Ohio for model training and evaluation. The training data set contains 1,392 images from the 249 cameras in Central Ohio, which are processed through an annotation process where bounding boxes of the vehicles are manually identified. The test data set consists of 465 traffic camera images in Central Ohio

This data set is combined by two external data sources, STREETS and WebCamT, to increase the size and diversity of the training data 

EfficientDet is a family of 8 object detection models (EfficientDet-d0 through d7) which share the same architecture. EfficientDet-d0 is the baseline model, and each of the next model has a larger size measured by its depth (number of layers), width (size of layers), and input image resolution. A large model size generally means high detection accuracy and high computational intensity.

In [ ]:
# Step 1: export saved model.
!python new_model_inspect.py --runmode=saved_model \
  --model_name=efficientdet-d3 --ckpt_path=model_dir/archive-d3 \
  --saved_model_dir=model_dir/saved_model \
  --hparams="image_size=1280x1024"

# Step 2: do inference with saved model.
!python new_model_inspect.py --runmode=saved_model_infer \
  --model_name=efficientdet-d3  \
  --saved_model_dir=model_dir/saved_model  \
  --input_image=data/buckeye_traffic/{imagedir}/*  \
  --output_image_dir=data/buckeye_traffic/{imagedir}_results/

<a id='hair'></a>
## Quadtree-based algorithm for HAIR identification

Unzip data from the sample camera.

In [4]:
from zipfile import ZipFile
with ZipFile('data/sample_cam.zip', 'r') as zipObj:
   zipObj.extractall('data')

<a id='ref'></a>
## References
<span id="fn1"> Tan, M., Pang, R., & Le, Q. V. (2020). Efficientdet: Scalable and efficient object detection. In Proceedings of the IEEE/CVF conference on computer vision and pattern recognition (pp. 10781-10790). </span>